# Containers communication

In this page I want to focus on ways of organising container communication.

## Creating containers

In the following cell, create containers for experiments.

In [1]:
%%bash
docker run --rm -d --name test_nginx1 nginx &> /dev/null
docker run --rm -d --name test_nginx2 nginx &> /dev/null
docker network create test_network &> /dev/null
docker run --rm -d \
    --name test_nginx3 \
    --net test_network \
    nginx &> /dev/null

**Note** Don't forget to stop the containers when you've finished playing with them.

In [ ]:
%%bash
docker stop test_nginx1 test_nginx2 test_nginx3 &> /dev/null
docker test_network

## Get ip of the containers

### Inspect network

There's a lot of important information in the output of `docker inspect` for network that you need to organise interaction between containers.

The following cell shows the output of `docker inspect` for network `bridge`. There you will find a `Containers` dictionary that matches the container id with information about the container. Among other things, the `IPv4Address` field is there.

In [2]:
%%bash
docker inspect bridge

[
    {
        "Name": "bridge",
        "Id": "cb0be6f8e70135dc9f75a03d73c9a5b9c9dd6225f18103191f676596edc9bdd1",
        "Created": "2023-10-30T07:56:28.229725692+03:00",
        "Scope": "local",
        "Driver": "bridge",
        "EnableIPv6": false,
        "IPAM": {
            "Driver": "default",
            "Options": null,
            "Config": [
                {
                    "Subnet": "172.17.0.0/16",
                    "Gateway": "172.17.0.1"
                }
            ]
        },
        "Internal": false,
        "Attachable": false,
        "Ingress": false,
        "ConfigFrom": {
            "Network": ""
        },
        "ConfigOnly": false,
        "Containers": {
            "63dae9a4f564ad5a4aa8654a57410eb02353fea7d0339aebd362ce4d893c6f46": {
                "Name": "test_nginx1",
                "EndpointID": "cbbae9a540e6fb959efda6c9f72a721e61c02f042243df2df76139e1b43d9ebe",
                "MacAddress": "02:42:ac:11:00:02",
                "IPv4

### Inspect container

The `docker inspect <container name/id>` command will give you information about the container, and by following the `NetworkSettings.IPAddress' path you can access the ip address of the container.

In [3]:
%%bash

echo "====nginx1 ip====="
echo\
    $(docker inspect\
    --format '{{ .NetworkSettings.IPAddress }}'\
    test_nginx1)

echo "====nginx2 ip====="
echo\
    $(docker inspect\
    --format '{{ .NetworkSettings.IPAddress }}'\
    test_nginx2)

====nginx1 ip=====
172.17.0.2
====nginx2 ip=====
172.17.0.3


## From container to container {#sec-from_cont_to_cont}

By using the IP address of the container, you can access it from the other container on the same network.

In the next cell, a commit is formed first, which will be called in the next step from the `test_nginx2` container to make a `curl` in `test_nginx1`. As a result we get html code which nginx responds to in its welcome page.

In [4]:
%%bash
# getting nginx2 ip
nginx2_ip=$(
    docker inspect \
    --format '{{ .NetworkSettings.IPAddress }}'\
    test_nginx2
)
command="curl -s $nginx2_ip"
echo "=====command to be executed====="
echo $command
echo
echo "=====execution result====="
docker exec test_nginx1 $command

=====command to be executed=====
curl -s 172.17.0.3

=====execution result=====
<!DOCTYPE html>
<html>
<head>
<title>Welcome to nginx!</title>
<style>
html { color-scheme: light dark; }
body { width: 35em; margin: 0 auto;
font-family: Tahoma, Verdana, Arial, sans-serif; }
</style>
</head>
<body>
<h1>Welcome to nginx!</h1>
<p>If you see this page, the nginx web server is successfully installed and
working. Further configuration is required.</p>

<p>For online documentation and support please refer to
<a href="http://nginx.org/">nginx.org</a>.<br/>
Commercial support is available at
<a href="http://nginx.com/">nginx.com</a>.</p>

<p><em>Thank you for using nginx.</em></p>
</body>
</html>


## Isolation example

The idea of networking in Docker is to be able to build effective structures as combinations of different containers. But you don't want containers that you don't expect to interact with to be able to interact. So in this case, you create containers that you don't want to interact with each other on a different network - you isolate them.

So in the following example, as in the [container-to-container](#sec-from_cont_to_cont) section, I'm trying to query from one container to another, but now I'm trying to access a container deployed on the other network and got an error.

In [10]:
%%bash
nginx2_ip=$(
    docker inspect \
    --format '{{ .NetworkSettings.IPAddress }}'\
    test_nginx2
)

command="curl -s $nginx2_ip"
echo "=====command to be executed====="
echo $command
echo
echo "=====execution result====="
docker exec test_nginx3 $command

=====command to be executed=====
curl -s 172.17.0.3

=====execution result=====


CalledProcessError: Command 'b'nginx2_ip=$(\n    docker inspect \\\n    --format \'{{ .NetworkSettings.IPAddress }}\'\\\n    test_nginx2\n)\n\ncommand="curl -s $nginx2_ip"\necho "=====command to be executed====="\necho $command\necho\necho "=====execution result====="\ndocker exec test_nginx3 $command\n'' returned non-zero exit status 28.